### This is the notebook for the final assignment

In [3]:
#include section
import numpy as np 
import pandas as pd
from pandas.io.html import read_html

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import requests 
from pandas.io.json import json_normalize 

!pip install pandas
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

     |████████████████████████████████| 5.8MB 8.6MB/s eta 0:00:01
     |████████████████████████████████| 112kB 6.0MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:



### Foursquare credentials

In [20]:
CLIENT_ID = 'FEAWVPXTSOFRAXM0FDHFXFNG4YT50L2ZOIEVU5WGAGVNEFZE' # your Foursquare ID
CLIENT_SECRET = '2RJP0MD4OA4LHTAXINVBCQFHCFMG2NH41H4NJRGAPIREGFPL' # your Foursquare Secret
VERSION = '20200430'
LIMIT = 30

Your credentails:
CLIENT_ID: FEAWVPXTSOFRAXM0FDHFXFNG4YT50L2ZOIEVU5WGAGVNEFZE
CLIENT_SECRET:2RJP0MD4OA4LHTAXINVBCQFHCFMG2NH41H4NJRGAPIREGFPL


### Creating a list of all districts of Vienna and getting the location via geocode

In [21]:
df = pd.DataFrame( columns = ['District', 'Latitude', 'Longitude']) 
df["District"] = ["Innere Stadt", "Leopoldstadt", "Landstraße", "Wieden", "Margareten", "Mariahilf", "Neubau", "Josefstadt", "Alsergrund", 
                  "Favoriten", "Simmering", "Meidling", "Hietzing", "Penzing", "Rudolfsheim-Fünfhaus", "Ottakring", "Hernals", "Währing", 
                  "Döbling", "Brigittenau", "Floridsdorf", "Donaustadt", "Liesing"]
df["Latitude"][3] = 1
geolocator = Nominatim(user_agent="foursquare_agent")
for i in range(0, 23):
    address = df["District"][i]+", Vienna"
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    df["Latitude"][i] = latitude
    df["Longitude"][i] = longitude
df

,District,Latitude,Longitude
0,Innere Stadt,48.209,16.3699
1,Leopoldstadt,48.2006,16.4269
2,Landstraße,48.1936,16.3963
3,Wieden,58.9089,14.7929
4,Margareten,48.1881,16.3534
5,Mariahilf,48.1955,16.347
6,Neubau,48.2023,16.3491
7,Josefstadt,48.2109,16.3474
8,Alsergrund,48.2251,16.3584
9,Favoriten,48.153,16.3828


### Create a map of Vienna with all districts

In [24]:
# create map of Vienna using latitude and longitude values
latitude = 48.2082
longitude = 16.3738
map_vienna = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, dis in zip(df['Latitude'], df['Longitude'], df['District']):
    label = '{}'.format(dis)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vienna)  
    
map_vienna

### Explore the venues of all Districts with a radius depending on the district size
### Bigger districts require a bigger radius otherwise we will not get enough venues

In [172]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
#The function to get all venues for each district
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        radius = 500
        if (name == "Donaustadt" or name == "Döbling" or name == "Favoriten" or name == "Hernals" or name == "Hietzing" 
            or name == "Leopoldstadt" or name == "Liesing" or name == "Ottakring" or name == "Simmering"):
            radius = 2000
 
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [173]:
#passing the district one by one to get the list of all venues
vienna_venues = getNearbyVenues(names=df['District'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Innere Stadt
Leopoldstadt
Landstraße
Wieden
Margareten
Mariahilf
Neubau
Josefstadt
Alsergrund
Favoriten
Simmering
Meidling
Hietzing
Penzing
Rudolfsheim-Fünfhaus
Ottakring
Hernals
Währing
Döbling
Brigittenau
Floridsdorf
Donaustadt
Liesing


In [174]:
print(vienna_venues.shape)
vienna_venues.head()

(933, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Innere Stadt,48.209023,16.369851,Stephansplatz,48.208299,16.371880,Plaza
1,Innere Stadt,48.209023,16.369851,Graben,48.208915,16.369379,Pedestrian Plaza
2,Innere Stadt,48.209023,16.369851,Stephansdom,48.208626,16.372672,Church
3,Innere Stadt,48.209023,16.369851,Kolar,48.210538,16.370033,Gastropub
4,Innere Stadt,48.209023,16.369851,COS,48.209359,16.371591,Clothing Store


In [175]:
#the number of vanues per district
vienna_groupby = vienna_venues.groupby('District').count().reset_index()
vienna_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Alsergrund,49,49,49,49,49,49
Brigittenau,12,12,12,12,12,12
Donaustadt,27,27,27,27,27,27
Döbling,63,63,63,63,63,63
Favoriten,41,41,41,41,41,41
Floridsdorf,11,11,11,11,11,11
Hernals,30,30,30,30,30,30
Hietzing,26,26,26,26,26,26
Innere Stadt,95,95,95,95,95,95


In [176]:
print('There are {} uniques categories.'.format(len(vienna_venues['Venue Category'].unique())))

There are 190 uniques categories.


### Analyze each district

In [177]:
# one hot encoding
vienna_onehot = pd.get_dummies(vienna_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
vienna_onehot['District'] = vienna_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [vienna_onehot.columns[-1]] + list(vienna_onehot.columns[:-1])
vienna_onehot = vienna_onehot[fixed_columns]

vienna_onehot.head()

,District,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,...,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Winery,Yoga Studio,Zoo Exhibit
0,Innere Stadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Innere Stadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Innere Stadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Innere Stadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Innere Stadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [178]:
vienna_onehot.shape

(933, 191)

In [179]:
vienna_grouped = vienna_onehot.groupby('District').mean().reset_index()
vienna_grouped

,District,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,...,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Winery,Yoga Studio,Zoo Exhibit
0,Alsergrund,0.000000,0.000000,0.000000,0.000000,0.040816,0.000000,0.020408,0.000000,0.020408,...,0.000000,0.000000,0.000000,0.040816,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000
1,Brigittenau,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Donaustadt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Döbling,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.015873,0.095238,0.000000,0.015873,0.000000,0.015873
4,Favoriten,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.121951,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Floridsdorf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.090909,0.000000,0.000000,0.000000
6,Hernals,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Hietzing,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.076923,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Innere Stadt,0.000000,0.010526,0.021053,0.000000,0.031579,0.000000,0.063158,0.000000,0.000000,...,0.010526,0.000000,0.010526,0.000000,0.000000,0.010526,0.000000,0.000000,0.010526,0.000000
9,Josefstadt,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.027778,0.000000,0.000000,...,0.013889,0.000000,0.000000,0.013889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [180]:
vienna_grouped.shape

(21, 191)

### Cluster Districts: I have decided to set the number of clusters to 4, equal to the 4 major political parties in Vienna

In [181]:
# set number of clusters
kclusters = 4

vienna_grouped_clustering = vienna_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(vienna_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_#[0:10] 


array([0, 2, 0, 3, 3, 1, 3, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0],
      dtype=int32)

In [183]:
# add clustering labels


# add clustering labels
vienna_groupby.insert(0, 'Cluster', kmeans.labels_)

vienna_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
vienna_merged = vienna_merged.join(vienna_groupby.set_index('District'), on='District')

#vienna_merged.head(25) # check the last columns!
#vienna_merged.loc[vienna_merged['Cluster Labels'] == "NaN"]
vienna_final = vienna_merged.loc[vienna_merged['Cluster'] >= 0]
vienna_final

,District,Latitude,Longitude,Cluster,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Innere Stadt,48.209,16.3699,0.0,95.0,95.0,95.0,95.0,95.0,95.0
1,Leopoldstadt,48.2006,16.4269,0.0,77.0,77.0,77.0,77.0,77.0,77.0
2,Landstraße,48.1936,16.3963,0.0,25.0,25.0,25.0,25.0,25.0,25.0
4,Margareten,48.1881,16.3534,0.0,27.0,27.0,27.0,27.0,27.0,27.0
5,Mariahilf,48.1955,16.347,0.0,50.0,50.0,50.0,50.0,50.0,50.0
6,Neubau,48.2023,16.3491,0.0,88.0,88.0,88.0,88.0,88.0,88.0
7,Josefstadt,48.2109,16.3474,0.0,72.0,72.0,72.0,72.0,72.0,72.0
8,Alsergrund,48.2251,16.3584,0.0,49.0,49.0,49.0,49.0,49.0,49.0
9,Favoriten,48.153,16.3828,3.0,41.0,41.0,41.0,41.0,41.0,41.0
10,Simmering,48.1631,16.458,0.0,41.0,41.0,41.0,41.0,41.0,41.0


In [185]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, dis, cluster in zip(vienna_final['Latitude'], vienna_final['Longitude'], vienna_final['District'], vienna_final['Cluster']):
    label = folium.Popup(str(dis) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color = "black", #color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters